In [5]:
lista = [(2,3), (3,5)]

In [50]:
%autoreload

UsageError: Line magic function `%autoreload` not found.


In [51]:
%load_ext autoreload


In [6]:
for elem, index in enumerate(lista):
    print('elem', elem)
    print(index)

elem 0
(2, 3)
elem 1
(3, 5)


In [5]:
tot=0
l = [(1,2), (2,3), (1,2)]
for child in l:
    print(child[0], child[1])
    count = 0
    for elem in lista:
        
        if child == elem:
        
            count = 1
            break
    print(count)
    if count == 0:
        tot +=1

1 2
0
2 3
1
1 2
0


In [1]:
import map


In [3]:
def GetMap():
    mapChar = []
    f = open('MAPA_LENDA-AANG.txt','r')
    lines = f.read().splitlines()
    for line in lines:
        lista = []
        for char in line:
            lista.append(char)
        mapChar.append(lista)
    return mapChar

In [4]:
mapa = GetMap()

In [ ]:
mapa[0]

In [8]:
v = np.zeros(4)

In [7]:
import numpy as np

In [9]:
v

array([0., 0., 0., 0.])

In [10]:
v[0]

0.0

In [11]:
v[0] = 2

In [12]:
v

array([2., 0., 0., 0.])

In [1]:
from algo import *

In [5]:
check_etapas = ['0', '1']
n = len(check_etapas)
check_points = positions_etapas(mapa, check_etapas, n)

In [9]:
check_etapas = ['0', '1']
n = len(check_etapas)
v = np.zeros(n, dtype=object)
for y in range(len(mapa)):
    for x in range(len(mapa[y])):
        for index, elem in enumerate(check_etapas):
            if mapa[y][x] == elem:
                v[index] = (x, y)
                print(index)
                print(y)
lista = list()
for i in range(n):
    print(v[i])
    # lista.append( (v[i][0], v[i][1]) )

1
56
0
75
(101, 75)
(101, 56)


In [1]:
def GetMap():
    mapChar = []
    f = open('MAPA_LENDA-AANG.txt','r')
    lines = f.read().splitlines()
    for line in lines:
        lista = []
        for char in line:
            lista.append(char)
        mapChar.append(lista)
    return mapChar

In [2]:
mapa = GetMap()

In [56]:
for index, elem in enumerate(check_etapas):
    print(index, elem)

0 0
1 1


In [3]:
len(mapa)

82

In [4]:
len(mapa[0])

300

In [5]:
mapa[x][y]

NameError: name 'x' is not defined

In [46]:
v = np.zeros(5)

In [47]:
v[0] = (0,1)

ValueError: setting an array element with a sequence.

In [40]:
check_etapas

['0', '1']

In [48]:
lista = []
lista.append(1)
lista.append(2)

In [49]:
lista

[1, 2]

In [2]:
from algo import *
from jogo import *

pygame 2.1.2 (SDL 2.0.18, Python 3.7.2)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
map = GetMap()
check_etapas = ['0', '1', '2', '3', '4', '5', '6', '7', '8',
                'B', 'C', 'D', 'E', 'G',
                'H', 'I', 'J', 'K', 'L',
                'N', 'O', 'P', 'Q', 'S', 
                'T', 'U', 'V', 'W', 'X', 'Y', 'Z']
n = len(check_etapas)
check_points = positions_etapas(map, check_etapas, n)

In [5]:
check_points[29]

(18, 44)

In [6]:
n

31

In [7]:
len(check_points)

31

In [9]:
for i in range(6):
    print(i)

0
1
2
3
4
5


In [12]:
check_points[30]

(30, 91)

In [17]:
def cria_novo_line(ncols, k):
    ltot = 0
    line = np.zeros(ncols)
    while ltot < k:
        n = randint(0,ncols-1)
        if line[n] != 1:
            line[n] = 1
            ltot += 1
    return line

In [18]:
from random import randint

In [21]:
a = cria_novo_line(7,4)

In [22]:
a[1] =2

In [23]:
a

array([0., 2., 1., 0., 1., 1., 0.])

In [24]:
a.sort()

In [25]:
a

array([0., 0., 0., 1., 1., 1., 2.])

In [26]:
lista = [1,2,3,0]

In [27]:
lista.sort()

In [28]:
lista

[0, 1, 2, 3]

In [30]:
a = ([2,34,5], 8)

In [31]:
a

([2, 34, 5], 8)

In [32]:
a[0]

[2, 34, 5]

In [33]:
type(a[0])

list

In [34]:
type(a[1])

int

In [4]:
import numpy as np

In [5]:
cria_novo_line(3,2)

NameError: name 'cria_novo_line' is not defined

In [6]:
def cria_novo_matrix(nlines, ncols, k):
    m=np.zeros((nlines, ncols))
    for i in range(nlines):
      ltot = 0
      while ltot < k:
         n = randint(0,ncols-1)
         if m[i][n] != 1:
               m[i][n] = 1
               ltot += 1
    return m

In [9]:
 m= cria_novo_matrix(4, 5, 3)
 m

array([[0., 1., 1., 1., 0.],
       [0., 1., 1., 0., 1.],
       [1., 0., 1., 1., 0.],
       [0., 1., 1., 1., 0.]])

In [8]:
from random import randint

In [15]:
m= cria_novo_matrix(4, 5, 3)
m

array([[0., 0., 1., 1., 1.],
       [0., 1., 1., 1., 0.],
       [0., 1., 1., 1., 0.],
       [0., 0., 1., 1., 1.]])

In [19]:
m[:, 0:2]

array([[0., 0.],
       [0., 1.],
       [0., 1.],
       [0., 0.]])

In [11]:
m

array([[0., 1., 1., 1., 0.],
       [0., 1., 1., 0., 1.],
       [1., 0., 1., 1., 0.],
       [0., 1., 1., 1., 0.]])

In [24]:
from genetico import *

In [25]:
n[:, 2:4] = np.ones(4)

NameError: name 'n' is not defined

In [28]:

# custo do tempo a cada etapa: dificuldade/ agilidade na etapa
def custo_tempo(etapa_dif, personagem_agilidade, matrix_genetica):
    matrix = np.divide(etapa_dif, np.matmul(matrix_genetica, personagem_agilidade))
    return np.sum(matrix)



def busca(p, lista): # returna indice i em que lista[i-1] < p < lista[i]
	low = 0
	high = len(lista) - 1
	while low <= high:
		middle = low + (high - low) // 2
		if lista[middle] >= p and lista[middle-1] < p: # lista[middle-1] < p <= lista[middle]
			return middle
		elif lista[middle] <= p:
			low = middle + 1 
		else:
			high = middle - 1
	return 0

def random_selection(population):
    etapa_dif = [3,4,5,100, 200, 300]
    personagem_agilidade = [1.2, 3, 4, 2]
    times = [custo_tempo(etapa_dif, personagem_agilidade, person) for person in population]
    times = np.divide(1, times) 
    probs = np.divide(times, np.sum(times)) ## mais provavel, menor tempos
    lista = []
    psum = 0
    n= len(probs)
    for i in range(n):
        psum += probs[i]
        lista.append(psum)
    p = random()
    j = busca(p, lista)
    return population[j] 


def reproduce(x,y):
    ## CUT THE COLUMNS
    etapa_dif = [3,4,5,100, 200, 300]
    personagem_agilidade = [1.2, 3, 4, 2]

    m = np.zeros((nlines, ncols))
    m[:, 0:3] = x[:, 0:3]
    m[:, 3:8] = y[:, 3:8]
    n = np.zeros((nlines, ncols))
    n[:, 0:3] = x[:, 0:3]
    n[:, 3:8] = y[:, 3:8]
    if checa_vazio(m) == -1:
        if checa_vazio(n) == -1:
            a = custo_tempo(etapa_dif, personagem_agilidade, m)
            b = custo_tempo(etapa_dif, personagem_agilidade, n)
            if a < b:
                return m
            else:
                return n
        else:
            return m
    else:
        if checa_vazio(n) == -1:
            return n
        else:
            # no child available
            # conserta?
            # send parents?
            a = custo_tempo(etapa_dif, personagem_agilidade, x)
            b = custo_tempo(etapa_dif, personagem_agilidade, y)
            if (a < b):
                return x
            return y
    

def best_individual(population):
    etapa_dif = [3,4,5,100, 200, 300]
    personagem_agilidade = [1.2, 3, 4, 2]


    times = [custo_tempo(etapa_dif, personagem_agilidade, person) for person in population]    
    best_time = times[0]
    index = 0
    for i in range(1, len(times)):
        if times[i] < best_time:
            best_time = times[i]
            index = i
    return best_time, population[index]
        
def genetic_algorithm(population,n): ## population list of matrices
    best_time, fit_ind = best_individual(population)
    print('best_time', best_time)
    for j in range(n):
        new_population = []
        # evaluate best individual...
        for i in range(len(population)):
            x = random_selection(population)
            y = random_selection(population)
            child = reproduce(x,y)
            p = random()
            # if (p < 0.01):
            #     child = mutate(child)
            new_population.append(child)
        population = new_population
        time, ind = best_individual(population)
        if time < best_time:
            best_time = time
            fit_ind = ind
            print('time', time)
            print('best_time', best_time)
    return fit_ind


In [12]:
population = list()
for i in range(4):
    population.append(cria_novo_matrix(6, 4,3))

In [13]:
population

[array([[0., 1., 1., 0.],
        [1., 0., 0., 0.],
        [1., 0., 1., 0.],
        [0., 1., 0., 1.],
        [1., 0., 0., 1.],
        [0., 1., 1., 1.]]),
 array([[0., 0., 0., 1.],
        [0., 1., 1., 0.],
        [1., 1., 0., 0.],
        [0., 0., 0., 1.],
        [1., 1., 1., 1.],
        [1., 0., 1., 0.]]),
 array([[1., 0., 1., 1.],
        [0., 0., 0., 0.],
        [0., 1., 0., 1.],
        [1., 0., 0., 0.],
        [0., 1., 1., 1.],
        [1., 1., 1., 0.]]),
 array([[0., 1., 0., 0.],
        [0., 1., 0., 1.],
        [0., 0., 0., 1.],
        [1., 0., 1., 0.],
        [1., 1., 1., 0.],
        [1., 0., 1., 1.]])]

In [24]:
m = population[0] 
m

array([[0., 1., 1., 0.],
       [1., 0., 0., 0.],
       [1., 0., 1., 0.],
       [0., 1., 0., 1.],
       [1., 0., 0., 1.],
       [0., 1., 1., 1.]])

In [25]:
m[0].sum()

2.0

In [17]:
nlines =  6
ncols = 4

In [35]:
genetic_algorithm(population, 5)

best_time 89.58767979987492
best_time 129.98534798534797
best_time 129.98534798534797
best_time 129.98534798534797
best_time 129.98534798534797
best_time 129.98534798534797


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in true_divide
  This is separate from the ipykernel package so we can avoid doing imports until


array([[0., 1., 1., 1.],
       [1., 0., 0., 0.],
       [1., 0., 1., 0.],
       [0., 1., 0., 1.],
       [1., 0., 0., 1.],
       [0., 1., 1., 0.]])

In [15]:
from random import random

In [33]:
m = np.matrix([[0,1], [1,1], [0,0]])
checa_vazio(*m)

soma 1
soma 2
soma 0


2

In [34]:
def checa_vazio(*m):    
    for i in range(len(m)):
        sum = m[i].sum()
        if sum == 0:
            return i
    return -1

def cria_novo_matrix(nlines, ncols, k):
    m=np.zeros((nlines, ncols))
    for i in range(ncols):
      ltot = 0
      while ltot < k:
         n = randint(0,nlines-1)
         if m[n][i] != 1:
               m[n][i] = 1
               ltot += 1
    index = checa_vazio(*m)
    while index != -1:
        n = randint(0,ncols-1)
        m[index][n] = 1
        s = -1
        ## atualizar matrix pois agora tem mais de k 1's na coluna
        k = randint(0, nlines-1)
        while m[k][n]== 0:
            k = randint(0, nlines-1)
        m[k][n] = 0
        index = checa_vazio(*m)
    return m 


In [84]:
n = np.zeros((4, 5))

In [95]:
n

array([[1., 1., 1., 0., 0.],
       [0., 1., 1., 0., 0.],
       [0., 0., 1., 0., 0.],
       [1., 1., 1., 0., 0.]])

In [82]:
m[:,3]

array([1., 0., 0., 1.])

In [66]:
np.append(m, [2])

array([2.])

In [67]:
m

array([], dtype=float64)

In [44]:
m[0]

array([1., 1., 1., 0., 0.])

In [47]:
lista = [[0,2], [1,1]]

In [50]:
lista[0][1]

2

In [51]:
lista[:,2]

TypeError: list indices must be integers or slices, not tuple